In [1]:
import skimage.io as io
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")  # skimage warns when saving uint16 with not covering full range of 0-65535
example_root = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 'examples')

## Train PCNA mitosis SVM classifier
### Author: Yifan Gui @ KuanYoow Chan group

After nuclear envelope breakdown, PCNA protein flees out into the cytoplasm, leading to dim signal intensity during mitosis. Additionally, the rounding cell are usually hard to be captured by the microscopy within focus limit. To reconstruct mitosis event from tracking results, pcnaDeep offers two algorithms: __spatial/temporal threshold-based assocation__ and __SVM-based classifier__. The first requires no training and totally depends on thresholding setting (see [Getting_started](Getting_started.ipynb)), while the second __SVM classifier__ requires a small training set and may perform better with large heterogenity of the cell.

### 1. Generate ground truth from model output
All features fed into pcnaDeep SVM classifier is extracted from the __tracked object table__, i.e., one of the standard outputs of the model ends with __`_tracks.csv`__.  Therefore, it is convinent to start with this file. In this tutorial, we will play with the file `example/MCF10A/10A_cpd_tracks.csv`.

_Note:_ __Do not__ start with refined table (ends with `_refined.csv`) where mitosis information has already been predicted.

In [4]:
dt = pd.read_csv(os.path.join(example_root, 'MCF10A', '10A_cpd_tracks.csv'))
dt

,frame,trackId,lineageId,parentTrackId,Center_of_the_object_1,Center_of_the_object_0,predicted_class,Probability of G1/G2,Probability of S,Probability of M,continuous_label,major_axis,minor_axis,mean_intensity
0,0,1,1,0,1051.247827,743.557187,G1/G2,0.999563,0.000219,0.000219,63,121.932617,104.651836,121.397962
1,1,1,1,0,1062.639710,754.025402,G1/G2,0.999718,0.000141,0.000141,67,123.190021,104.308464,117.046041
2,2,1,1,0,1067.604006,752.252132,G1/G2,0.999568,0.000216,0.000216,70,123.480678,104.168394,120.547194
3,3,1,1,0,1073.626605,749.558346,G1/G2,0.998859,0.000571,0.000571,71,117.147045,96.619333,105.931291
4,4,1,1,0,1071.141247,757.504600,G1/G2,0.995962,0.002019,0.002019,69,114.777681,85.751799,94.643644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,64,137,137,0,504.549438,14.979837,G1/G2,0.998686,0.000657,0.000657,30,90.525362,38.989439,199.423808
5045,65,137,137,0,504.329663,17.151151,G1/G2,0.998191,0.000905,0.000905,29,92.580038,43.857111,189.004338
5046,63,138,138,0,513.379839,1033.807258,G1/G2,0.751172,0.124414,0.124414,33,59.738632,53.146233,66.047984
5047,64,138,138,0,519.049538,1048.395861,M,0.068280,0.068280,0.863440,31,83.341466,69.747563,73.680097


### &emsp;1.1 Generate mitosis-breaked table
pcnaDeep SVM takes in inputs extracted from __track A__ and __track B__ and outputs __the confidence that track A is the parent of track B__. This requires each track does not includes any entire mitosis events, which is accomplished in the first step of normal track refinement by a class method of `pcnaDeep.refiner.Refiner` named __`break_mitosis( )`__, which implements a greedy search throughout the track. Such operation should be recursively performed to break any possible mitosis events.

The above process can be automatically completed through calling `pcnaDeep.refiner.Refiner.doTrackRefine( )` in a `Refiner` object initialized with `TRAIN` mode. The method returns the tracked object table with all mitosis broken.

In [7]:
from pcnaDeep.refiner import Refiner
r = Refiner(dt.copy(), mode='TRAIN')
mitosis_broken = r.doTrackRefine()

Object classification corrected by smoothing: 169
Found mitosis track: 30
Found mitosis track: 3
Found mitosis track: 0
High quality tracks subjected to predict relationship: 129


We must __save the output table__ for visualization in the next step.

In [10]:
mitosis_broken.to_csv(os.path.join(example_root, 'MCF10A', '10A_cpd_tracks_mt_broken.csv'))

### &emsp;1.2 Visualize in Fiji (ImageJ) to inspect mitosis events

_Note:_ Make sure you have already installed [__TrackMate CSV Importer__](https://github.com/tinevez/TrackMate-CSVImporter) Fiji plugin.

1. Load `exmaple/MCF10A/10A_cpd.tif` to Fiji through dragging it to the tool bar or __File > Open__.


2. Invert t and z axis of the image if necessary, through __Image > Properties__. In this example, set _frame (t)_ to 66 and _slice (z)_ to 1.


3. Open TrackMate CSV Importer through __Plugin > Tracking > TrackMate CSV Importer__.


4. Config parameters as the following (see screenshot 👉 ):
    <img src="assets/trackMate_guide_1.png"  alt="config trackmate csv importer" width=450 align="right"/>
    
    - Compute all features? = unchecked; Import tracks? = tracked
    - Radius: choose one comfortable to the cell size, in this case is 50 pixels
    - X/Y column: Center_of_the_object_0/1
    - Z column: "Don't use"
    - Frame column: frame
    - Track column: trackId
    - Name column: trackId
    - Quality column & ID column: "Don't use"
    
    
5. Click 'Import', and then adjust GUI
    - check "Display spot names"
    - under "Track display mode", choose "Show local tracks"
    
    
6. Identify mitosis events, record __parent and daughter tracks__ in a `.txt` file.   
   To plus, only positive sample is not enough for model training, therfore, __randomly record some negative pairs__ (i.e., track A and track B are not parent and daughter). The sample should be balanced.  
   
    Format (header not required, from example view below 👇):
    | parent | daughter| 0/1 <br> (positive/negative)|
    | :---: | :---: | :---: |
    | 140 | 169 | 1 |
    | 140 | 121 | 1 |
    | 140 | 122 | 0 |

    ---
    
    <img src="assets/trackMate_guide_2.png"  alt="mitosis example" width=900/>
    
    ---
    
     __Some rules to follow__:
    - If a mitosis track is not broken (e.g. in the above picture, 141 and its daughter 122), this is a prediction error from function call `break_mitosis( )`. This can be overcome by adjusting threshold of mitosis greedy search. However, when generating ground truth, there is no need to consider these cases, i.e., do not put (141, 122, 1) in the table, but (141, 122, 0) is fine.
    - When generating negative instances, try to 'challenge' our classifier with difficult questions. For example, track 122 is very near track 140 though not a daughte of it. Challenge the classifier by putting (140, 122, 0) in your table.
    
    

### 2. (Alternative to 1) Generate ground truth from deepcell .trk gorund truth

### 3. Initialize, Fit, Save the SVM model